# Проект по SQL

## Введение

Из-за эпидемии под названием "коронавирус" в свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры, но стало больше времени для книг. Исходя из данного факта, наша команда решила купить крупный сервис для чтения книг по подписке. 

**Цель исследования:** проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта

**Ход исследования:** 
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подключение к базе данных

*Перед началом исследования подготовим подключения к базе:*

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

    Результат проделанной работы: Данные готовы к дальнейшему исследованию!

## Посчитаем, сколько книг вышло после 1 января 2000 года

In [2]:
query = ''' SELECT COUNT(title)
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


    Результат: книг вышло после 1 января 2000 года - 819

## Для каждой книги посчитаем количество обзоров и среднюю оценку

In [59]:
query = ''' SELECT b.title AS title, b.book_id, COUNT(DISTINCT reviews.text) AS count, ROUND(AVG(ratings.rating), 2)
FROM books AS b

INNER JOIN ratings ON ratings.book_id = b.book_id
INNER JOIN reviews ON reviews.book_id = b.book_id

GROUP BY b.title, b.book_id     
ORDER BY count DESC
'''
pd.io.sql.read_sql(query, con = engine)

,title,book_id,count,round
0,Twilight (Twilight #1),948,7,3.66
1,The Road,854,6,3.77
2,The Book Thief,656,6,4.26
3,The Glass Castle,734,6,4.21
4,Water for Elephants,963,6,3.98
...,...,...,...,...
989,Naked Empire (Sword of Truth #8),465,1,3.50
990,Moo Baa La La La!,446,1,3.00
991,Merrick (The Vampire Chronicles #7),431,1,4.00
992,Babyville,92,1,3.50


    Результат: для каждой книги было посчитано количество обзоров и средняя оценка

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [4]:
query = ''' SELECT publisher, COUNT(title)
FROM publishers AS p
LEFT JOIN books as b ON p.publisher_id = b.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY count DESC
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


    Результат: определили издательство, которое выпустило наибольшее число книг при их толщине более 50 страниц - это "Penguin Books"

## Определим автора с самой высокой средней оценкой книг (с 50 и более оценками)

In [5]:
query = ''' SELECT
      a.author, AVG(a.avg_rating) AS avg_rating
FROM 
     (SELECT author, b.book_id AS id, AVG(rating) AS avg_rating, COUNT(rating) AS cnt_rating
      FROM
      authors AS au
      
      LEFT JOIN books AS b ON b.author_id = au.author_id
      LEFT JOIN ratings AS r ON r.book_id = b.book_id
      
      GROUP BY
      author, id
      HAVING
      COUNT(rating) >= 50) AS a
      
GROUP BY
    author
ORDER BY
    avg_rating DESC
    
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


    Результат: определили автора с самой высокой средней оценкой книг (с 50 и более оценками) - J.K. Rowling/Mary GrandPré. Средняя оценка ее книг составляет  - 4.284

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [6]:
query = ''' SELECT ROUND(avg(count)) 
FROM (SELECT COUNT(text)
     FROM reviews AS r
     LEFT JOIN (SELECT username, COUNT(rating_id)
                FROM ratings
                GROUP BY username) AS r1 ON r1.username = r.username
     WHERE count > 50
     GROUP BY r.username) AS count                                  
'''

pd.io.sql.read_sql(query, con = engine)

,round
0,24.0


    Результат: Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24

## Общий вывод

Проанализировав базу данных, вывод получился слежующим:
- Книг вышло после 1 января 2000 года - 819;
- Для каждой книги было посчитано количество обзоров и средняя оценка;
- Определили издательство, которое выпустило наибольшее число книг при их толщине более 50 страниц - это "Penguin Books";
- Определили автора с самой высокой средней оценкой книг (с 50 и более оценками) - J.K. Rowling/Mary GrandPré. Средняя оценка ее книг составляет  - 4.284 из 5 возможных;
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24<br><br>

**Вышеперечисленные данные помогут сформулировать ценностное предложение для нового продукта!**